In [43]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import s327277  

# -----------------------------
# Configuration
# -----------------------------
PROBLEM = 8
SEED = 42
TEST_SIZE = 1 

# -----------------------------
# Load function and data
# -----------------------------
function = getattr(s327277, f"f{PROBLEM}")
data = np.load(f"./data/problem_{PROBLEM}.npz")
x_data = data['x']
y_data = data['y']
print(f"Evaluating problem {PROBLEM}: x shape = {x_data.shape}, y shape = {y_data.shape}")

# -----------------------------
# Train/test split
# -----------------------------
np.random.seed(SEED)
dataset_size = x_data.shape[1]
train_idx, val_idx = train_test_split(range(dataset_size), test_size=TEST_SIZE, random_state=SEED)

x_train, y_train = x_data[:, train_idx], y_data[train_idx]
x_val, y_val = x_data[:, val_idx], y_data[val_idx]

# -----------------------------
# Evaluate performance
# -----------------------------
def mse(y_pred, y_true):
    return 100 * np.square(y_pred - y_true).sum() / len(y_true)

y_pred_full = function(x_data)
y_pred_train = function(x_train)
y_pred_val = function(x_val)

print(f"\nMSE (overall): {mse(y_pred_full, y_data):.6g}")
print(f"MSE (train)  : {mse(y_pred_train, y_train):.6g}")
print(f"MSE (val)    : {mse(y_pred_val, y_val):.6g}")

# -----------------------------
# Create output folder
# -----------------------------
result_dir = f"./results/problems/problem{PROBLEM}"
os.makedirs(result_dir, exist_ok=True)

# -----------------------------
# 2D plots: one for each variable
# -----------------------------
def plot_1d_solution(x, y, f, i):
    domain = np.zeros((x.shape[0], 1000))
    domain[i] = np.linspace(x[i].min(), x[i].max(), 1000)
    prediction = f(domain)

    plt.figure(figsize=(6, 4))
    plt.scatter(x[i], y, color='red', s=2, label="Actual")
    plt.plot(domain[i], prediction, color='blue', label="Predicted")
    plt.xlabel(f"x[{i}]")
    plt.ylabel("y")
    plt.title(f"Problem {PROBLEM} - Feature x[{i}]")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{result_dir}/x{i}.png")
    plt.close()

for i in range(x_data.shape[0]):
    plot_1d_solution(x_data, y_data, function, i)

# -----------------------------
# 3D surface plot if input is 2D
# -----------------------------
if x_data.shape[0] == 2:
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    x0 = np.linspace(x_data[0].min(), x_data[0].max(), 100)
    x1 = np.linspace(x_data[1].min(), x_data[1].max(), 100)
    X0, X1 = np.meshgrid(x0, x1)
    grid = np.array([X0.ravel(), X1.ravel()])
    Z = function(grid).reshape(X0.shape)

    ax.plot_surface(X0, X1, Z, alpha=0.6, cmap='viridis')
    ax.scatter(x_data[0], x_data[1], y_data, c='r', s=1)

    ax.set_xlabel("x[0]")
    ax.set_ylabel("x[1]")
    ax.set_zlabel("y")
    ax.set_title(f"Problem {PROBLEM} - 3D Surface")
    plt.tight_layout()
    plt.savefig(f"{result_dir}/3d.png")
    plt.close()


if x_data.shape[0] == 3:  # Three input variables: x0, x1, x2
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Create a grid for x0 and x1
    x0 = np.linspace(x_data[0].min(), x_data[0].max(), 100)
    x1 = np.linspace(x_data[1].min(), x_data[1].max(), 100)
    X0, X1 = np.meshgrid(x0, x1)

    # Fix x2 at its median value
    x2_fixed = np.median(x_data[2])
    X2 = np.full_like(X0, x2_fixed)

    # Create grid with shape (3, N) for evaluation
    grid = np.array([X0.ravel(), X1.ravel(), X2.ravel()])

    # Evaluate model on the grid
    Z = function(grid).reshape(X0.shape)

    # Plot surface and original samples
    ax.plot_surface(X0, X1, Z, alpha=0.6, cmap='viridis')
    ax.scatter(x_data[0], x_data[1], y_data, c='r', s=1)

    ax.set_xlabel("x[0]")
    ax.set_ylabel("x[1]")
    ax.set_zlabel("y")
    ax.set_title(f"Problem {PROBLEM} - 3D Slice with x[2] = {x2_fixed:.2f}")
    plt.tight_layout()
    plt.savefig(f"{result_dir}/3d.png")
    plt.close()




print(f"\nAll results saved to: {result_dir}")







Evaluating problem 8: x shape = (6, 50000), y shape = (50000,)

MSE (overall): 1.1711e+08
MSE (train)  : 1.1711e+08
MSE (val)    : 1.51004e+08

All results saved to: ./results/problems/problem8
